In [4]:
import os
import uuid
import shutil
#import pylatex
import pandas as pd
import lightkurve as lk 
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

from datetime import datetime
from tqdm.notebook import tqdm
from multiprocessing import cpu_count, Pool
#from pylatex.section import Chapter, Subsection
#from pylatex import Document, Section, Figure, NoEscape, Command

%run pipeline.ipynb

import warnings
warnings.filterwarnings("ignore")

In [5]:
# bestlcs=pd.read_csv('makingtable/bestlcs.csv')
# bestlclist=np.array([], dtype=object)
# start_col=bestlcs.columns.get_loc('Sector 1')
# for index, row in bestlcs.iterrows(): #iterate through each object
#     for col in bestlcs.columns[start_col:]: #iterate through all the columns starting with the sectors
#         value=row[col]
#         if not pd.isna(value):
#             bestlclist=np.append(bestlclist, [f'{row['pop_id']}_{value}'])

In [6]:
# K2SFFlist=np.loadtxt('makingtable/K2SFFlist.txt', dtype=str)
# lclist=pd.concat([pd.DataFrame(bestlclist), pd.DataFrame(K2SFFlist)], ignore_index=True)
# lclist[['pop_id', 'lcname']]=lclist[0].str.split('_', expand=True)
# lclist.drop(columns=0, inplace=True)

In [7]:
#lclist=pd.read_csv('periodtable.csv')

In [8]:
#lclist['complex_status']=""

In [9]:
lclist=pd.read_csv('lcnameswithstatus.csv')

In [10]:
for index, currow in lclist.iterrows(): 
    if not pd.isna(currow['complex_status']):
        continue
    try:
        curlc=get_lc_string(currow['lcname'])
        complexity,period= harmonic_checker(curlc,max_period=6,percentage=0.10)
        lclist.at[index,'complex_status']=complexity
        lclist.at[index,'calc_per']=period
    except Exception as e:
        print(f'Error getting complex status for {currow['lcname']}: {e}')

Error getting complex status for TIC-404335106-38-FFI-10min: [Errno 60] Operation timed out
Error getting complex status for TIC-300651846-13-FFI-30min: index -1 is out of bounds for axis 0 with size 0
Error getting complex status for TIC-442571495-91-FFI-200s: index -1 is out of bounds for axis 0 with size 0
Error getting complex status for TIC-68167990-91-FFI-200s: cannot unpack non-iterable NoneType object
Error getting complex status for TIC-442575691-91-FFI-200s: index -1 is out of bounds for axis 0 with size 0
Error getting complex status for TIC-85125492-39-FFI-10min: cannot unpack non-iterable NoneType object


No data found for target "EPIC 60019763".


Error getting complex status for EPIC-60019763-1-K2SFF-1800: 'NoneType' object has no attribute 'to_periodogram'
Error getting complex status for TIC-244161191-42-FFI-10min: index -1 is out of bounds for axis 0 with size 0


In [11]:
np.unique(lclist[pd.isna(lclist.complex_status)]['author_x'],return_counts=True)

(array(['FFI', 'K2SFF'], dtype=object), array([7, 1]))

In [12]:
np.unique(lclist[np.logical_not(pd.isna(lclist.complex_status))]['author_x'],return_counts=True)

(array(['EVEREST', 'FFI', 'K2SFF', 'SPOC'], dtype=object),
 array([ 57, 419,  57, 443]))

In [13]:
lclist.sort_values(by=['pop_id','idtype','secorcamp'],inplace=True)

In [33]:
lclist['epic_id']=lclist['epic_id'].astype(pd.Int64Dtype()).astype(str)

In [35]:
lclist

,pop_id,lcname,idtype,ID,secorcamp,author_x,cadence,fluxamplitude,TIC,gaiadr3_source_id,epic_id,per,per2,group,age_Myr,disco_paper,author_y,year,calc_per,complex_status
0,0,EPIC-246676629-13-EVEREST-1800,EPIC,246676629,13,EVEREST,1800,0.075449,59129133,3.392549e+18,246676629,0.6253,0.6332,Taurus,2.0,Stauffer_2018,Stauffer,2018,0.625354,Complex
1,0,EPIC-246676629-13-K2SFF-1800,EPIC,246676629,13,K2SFF,1800,0.083411,59129133,3.392549e+18,246676629,0.6253,0.6332,Taurus,2.0,Stauffer_2018,Stauffer,2018,0.625474,Complex
2,0,TIC-59129133-5-FFI-30min,TIC,59129133,5,FFI,30min,0.041935,59129133,3.392549e+18,246676629,0.6253,0.6332,Taurus,2.0,Stauffer_2018,Stauffer,2018,0.628135,Complex
3,0,TIC-59129133-32-FFI-10min,TIC,59129133,32,FFI,10min,0.053793,59129133,3.392549e+18,246676629,0.6253,0.6332,Taurus,2.0,Stauffer_2018,Stauffer,2018,0.628736,Complex
4,0,TIC-59129133-43-SPOC-120,TIC,59129133,43,SPOC,120,0.053993,59129133,3.392549e+18,246676629,0.6253,0.6332,Taurus,2.0,Stauffer_2018,Stauffer,2018,0.627166,Complex
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
979,211,EPIC-247794636-13-K2SFF-1800,EPIC,247794636,13,K2SFF,1800,0.230866,268324578,1.477992e+17,247794636,1.7211,2.6225,Taurus,2.0,Rebull_2020,Rebull,2020,2.257125,Complex
980,211,TIC-268324578-43-SPOC-120,TIC,268324578,43,SPOC,120,0.153299,268324578,1.477992e+17,247794636,1.7211,2.6225,Taurus,2.0,Rebull_2020,Rebull,2020,2.220887,Double Dip
981,211,TIC-268324578-44-SPOC-120,TIC,268324578,44,SPOC,120,0.122678,268324578,1.477992e+17,247794636,1.7211,2.6225,Taurus,2.0,Rebull_2020,Rebull,2020,2.221172,Not Complex
982,211,TIC-268324578-70-FFI-200s,TIC,268324578,70,FFI,200s,0.015121,268324578,1.477992e+17,247794636,1.7211,2.6225,Taurus,2.0,Rebull_2020,Rebull,2020,2.281190,Double Dip


In [16]:
id_table=pd.read_csv('makingtable/idswithdiscopaper.csv',dtype={'gaiadr3_source_id':str})

In [39]:
id_table=id_table[['pop_id','gaiadr3_source_id']]

In [41]:
merged=pd.merge(lclist,id_table,on='pop_id',how='left')

In [45]:
merged.gaiadr3_source_id_x=merged.gaiadr3_source_id_y

In [49]:
merged.drop('gaiadr3_source_id_y',axis=1,inplace=True)

In [53]:
merged.rename(columns={'gaiadr3_source_id_x':'gaiadr3_source_id'},inplace=True)

In [57]:
merged.to_csv('lcnameswithstatus.csv',index=False)

In [55]:
merged

,pop_id,lcname,idtype,ID,secorcamp,author_x,cadence,fluxamplitude,TIC,gaiadr3_source_id,epic_id,per,per2,group,age_Myr,disco_paper,author_y,year,calc_per,complex_status
0,0,EPIC-246676629-13-EVEREST-1800,EPIC,246676629,13,EVEREST,1800,0.075449,59129133,3392549449695395968,246676629,0.6253,0.6332,Taurus,2.0,Stauffer_2018,Stauffer,2018,0.625354,Complex
1,0,EPIC-246676629-13-K2SFF-1800,EPIC,246676629,13,K2SFF,1800,0.083411,59129133,3392549449695395968,246676629,0.6253,0.6332,Taurus,2.0,Stauffer_2018,Stauffer,2018,0.625474,Complex
2,0,TIC-59129133-5-FFI-30min,TIC,59129133,5,FFI,30min,0.041935,59129133,3392549449695395968,246676629,0.6253,0.6332,Taurus,2.0,Stauffer_2018,Stauffer,2018,0.628135,Complex
3,0,TIC-59129133-32-FFI-10min,TIC,59129133,32,FFI,10min,0.053793,59129133,3392549449695395968,246676629,0.6253,0.6332,Taurus,2.0,Stauffer_2018,Stauffer,2018,0.628736,Complex
4,0,TIC-59129133-43-SPOC-120,TIC,59129133,43,SPOC,120,0.053993,59129133,3392549449695395968,246676629,0.6253,0.6332,Taurus,2.0,Stauffer_2018,Stauffer,2018,0.627166,Complex
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
979,211,EPIC-247794636-13-K2SFF-1800,EPIC,247794636,13,K2SFF,1800,0.230866,268324578,147799209159857280,247794636,1.7211,2.6225,Taurus,2.0,Rebull_2020,Rebull,2020,2.257125,Complex
980,211,TIC-268324578-43-SPOC-120,TIC,268324578,43,SPOC,120,0.153299,268324578,147799209159857280,247794636,1.7211,2.6225,Taurus,2.0,Rebull_2020,Rebull,2020,2.220887,Double Dip
981,211,TIC-268324578-44-SPOC-120,TIC,268324578,44,SPOC,120,0.122678,268324578,147799209159857280,247794636,1.7211,2.6225,Taurus,2.0,Rebull_2020,Rebull,2020,2.221172,Not Complex
982,211,TIC-268324578-70-FFI-200s,TIC,268324578,70,FFI,200s,0.015121,268324578,147799209159857280,247794636,1.7211,2.6225,Taurus,2.0,Rebull_2020,Rebull,2020,2.281190,Double Dip
